In [ ]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras import models

caltech_dir = './Data/Apple_one/Train'
categories = ['BadApple', 'GoodApple','UgglyApple'] #디렉토리명
nb_classes = len(categories)

image_w = 64
image_h =64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./Data/Numpy_data/Cnn_Apple_one_data.npy", xy) # 넘파이 파일로 저장

print("ok", len(y))

#넘파이 파일 로드 후 분리
X_train, X_test, y_train, y_test = np.load('./Data/Numpy_data/Cnn_Apple_one_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

categories = ['BadApple', 'GoodApple','UgglyApple']
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

model = models.Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=X_train.shape[1:],padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (3,3), activation='relu',padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (3,3), activation='relu',padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

checkpoint = keras.callbacks.ModelCheckpoint('./model/CNN_one.h5', save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=80,batch_size=32,
                     validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()